# Pokémon Classification with Deep Learning Applications
*   Mert Çalışkan
*   Zübeyir Faruk Tekbaş

## Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import random
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization, Activation, LeakyReLU, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.regularizers import l1, l2, l1_l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Input
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.applications import VGG16
from math import ceil


In [ ]:
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)


## Data Set Reading

In [ ]:
data_dir = 'images'
csv_file = 'pokemon.csv'
pokemon_data = pd.read_csv(csv_file)
pokemon_data['image_path'] = pokemon_data['Name'].apply(lambda x: f'{data_dir}/{x}.png')
pokemon_data['label'] = pokemon_data['Type1'].astype('category').cat.codes


## Data Analysis and Pre-processing

In [ ]:
# Check for missing values
print(pokemon_data.isnull().sum())

pokemon_data.drop(columns=["Type2", "Evolution"], inplace=True)


In [ ]:
pokemon_data['image'] = pokemon_data['image_path'].apply(lambda path: img_to_array(load_img(path, target_size=(120, 120))) / 255.0)
images = np.stack(pokemon_data['image'].values)
labels = to_categorical(pokemon_data['label'].values)
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
# Decoding
def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_train_original = decode_labels(np.argmax(y_train, axis=1), pokemon_data)
y_val_original = decode_labels(np.argmax(y_val, axis=1), pokemon_data)
y_test_original = decode_labels(np.argmax(y_test, axis=1), pokemon_data)

pokemon_data['Type1_decoded'] = decode_labels(pokemon_data['label'], pokemon_data)


In [ ]:
print(y_train.shape)
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)


In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

batch_size = 32
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)


In [ ]:
print(pokemon_data['Type1'].value_counts())


In [ ]:
# Count the values in the 'Type1' column
type_counts = pokemon_data['Type1'].value_counts()

sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.barplot(x=type_counts, y=type_counts.index, palette="viridis")
plt.xlabel('Pokémon Count')
plt.ylabel('Type1')
for i, count in enumerate(type_counts):
    plt.text(count, i, f' {count}', va='center')
plt.grid(False)
plt.show()


In [ ]:
num_images = 16
random_indices = np.random.choice(len(images), num_images, replace=False)
fig, axes = plt.subplots(2, 8, figsize=(12, 4))
for ax, idx in zip(axes.flat, random_indices):
    ax.imshow(images[idx])
    ax.axis('off')
    ax.set_title(f"{pokemon_data['Type1'][idx]}")
plt.tight_layout()
plt.show()


## MLP Models

In [ ]:
results = []


### MLP Basic

In [ ]:
model_mlp_basic = Sequential([
    Input(shape=(120, 120, 3)),  # Giriş katmanı
    Flatten(),
    Dense(512),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(len(np.unique(pokemon_data['Type1'])), activation='softmax') 
])

model_mlp_basic.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_mlp_basic.summary()

# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-8, verbose=1)
callbacks = [early_stopping, reduce_lr]

history_mlp_basic = model_mlp_basic.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=callbacks
)


In [ ]:
train_acc = history_mlp_basic.history['accuracy']
val_acc = history_mlp_basic.history['val_accuracy']
train_loss = history_mlp_basic.history['loss']
val_loss = history_mlp_basic.history['val_loss']

epochs = range(1, len(train_acc) + 1)

test_loss, test_acc = model_mlp_basic.evaluate(X_test, y_test)
print(f'Training Accuracy: {train_acc[-1]}')
print(f'Validation Accuracy: {val_acc[-1]}')
print(f'Test Accuracy: {test_acc}')
print(f'Training Loss: {train_loss[-1]}')
print(f'Validation Loss: {val_loss[-1]}')
print(f'Test Loss: {test_loss}')


In [ ]:
sns.set(style="whitegrid")

# Training and Validation Accuracy Plot
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(visible=None)
plt.show()

# Training and Validation Loss Plot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(visible=None)
plt.show()


In [ ]:
# Evaluation metrics
y_pred_mlp_basic = model_mlp_basic.predict(X_test)
y_pred_classes_mlp_basic = np.argmax(y_pred_mlp_basic, axis=1)
y_true_mlp_basic = np.argmax(y_test, axis=1)
accuracy_mlp_basic = accuracy_score(y_true_mlp_basic, y_pred_classes_mlp_basic)
precision_mlp_basic = precision_score(y_true_mlp_basic, y_pred_classes_mlp_basic, average='weighted', zero_division=0)
recall_mlp_basic = recall_score(y_true_mlp_basic, y_pred_classes_mlp_basic, average='weighted')
f1_mlp_basic = f1_score(y_true_mlp_basic, y_pred_classes_mlp_basic, average='weighted')
results.append({'Model': 'MLP Basic', 'Accuracy': accuracy_mlp_basic, 'Precision': precision_mlp_basic, 'Recall': recall_mlp_basic, 'F1 Score': f1_mlp_basic})


In [ ]:
y_pred_probs = model_mlp_basic.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_pred_labels = decode_labels(y_pred_classes, pokemon_data)
y_true_labels = decode_labels(y_true_classes, pokemon_data)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true_labels), yticklabels=np.unique(y_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


### MLP1

In [ ]:
model_mlp1 = Sequential([
    Input(shape=(120, 120, 3)),
    Flatten(),
    Dense(32, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), kernel_initializer=HeNormal()),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), kernel_initializer=HeNormal()),
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(len(np.unique(pokemon_data['Type1'])), activation='softmax') 
])

model_mlp1.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_mlp1.summary()

# Callback 
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-8, verbose=1)
callbacks = [early_stopping, reduce_lr]

history_mlp1 = model_mlp1.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=callbacks
)


In [ ]:
train_acc = history_mlp1.history['accuracy']
val_acc = history_mlp1.history['val_accuracy']
train_loss = history_mlp1.history['loss']
val_loss = history_mlp1.history['val_loss']

epochs = range(1, len(train_acc) + 1)

test_loss, test_acc = model_mlp1.evaluate(X_test, y_test)
print(f'Training Accuracy: {train_acc[-1]}')
print(f'Validation Accuracy: {val_acc[-1]}')
print(f'Test Accuracy: {test_acc}')
print(f'Training Loss: {train_loss[-1]}')
print(f'Validation Loss: {val_loss[-1]}')
print(f'Test Loss: {test_loss}')


In [ ]:
sns.set(style="whitegrid")

# Training and Validation Accuracy Plot
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(visible=None)
plt.show()

# Training and Validation Loss Plot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(visible=None)
plt.show()


In [ ]:
y_pred_mlp1 = model_mlp1.predict(X_test)
y_pred_classes_mlp1 = np.argmax(y_pred_mlp1, axis=1)
y_true_mlp1 = np.argmax(y_test, axis=1)
accuracy_mlp1 = accuracy_score(y_true_mlp1, y_pred_classes_mlp1)
precision_mlp1 = precision_score(y_true_mlp1, y_pred_classes_mlp1, average='weighted', zero_division=0)
recall_mlp1 = recall_score(y_true_mlp1, y_pred_classes_mlp1, average='weighted')
f1_mlp1 = f1_score(y_true_mlp1, y_pred_classes_mlp1, average='weighted')
results.append({'Model': 'MLP1', 'Accuracy': accuracy_mlp1, 'Precision': precision_mlp1, 'Recall': recall_mlp1, 'F1 Score': f1_mlp1})


In [ ]:
y_pred_probs = model_mlp1.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_pred_labels = decode_labels(y_pred_classes, pokemon_data)
y_true_labels = decode_labels(y_true_classes, pokemon_data)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true_labels), yticklabels=np.unique(y_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


### MLP2

In [ ]:
model_mlp2 = Sequential([
    Input(shape=(120, 120, 3)),
    Flatten(),
    Dense(32, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), kernel_initializer=HeNormal()),  
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), kernel_initializer=HeNormal()), 
    Activation('relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(len(np.unique(pokemon_data['Type1'])), activation='softmax')
])

model_mlp2.compile(optimizer=RMSprop(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_mlp2.summary()

# Define Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-8, verbose=1)
callbacks = [early_stopping, reduce_lr]

# Start the Training
history_mlp2 = model_mlp2.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=callbacks
)


In [ ]:
train_acc = history_mlp2.history['accuracy']
val_acc = history_mlp2.history['val_accuracy']
train_loss = history_mlp2.history['loss']
val_loss = history_mlp2.history['val_loss']

# List containing epoch numbers
epochs = range(1, len(train_acc) + 1)

# Calculate and print test accuracy and loss
test_loss, test_acc = model_mlp2.evaluate(X_test, y_test)
print(f'Training Accuracy: {train_acc[-1]}')
print(f'Validation Accuracy: {val_acc[-1]}')
print(f'Test Accuracy: {test_acc}')
print(f'Training Loss: {train_loss[-1]}')
print(f'Validation Loss: {val_loss[-1]}')
print(f'Test Loss: {test_loss}')


In [ ]:
sns.set(style="whitegrid")

# Training and Validation Accuracy Plot
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(visible=None)
plt.show()

# Training and Validation Loss Plot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(visible=None)
plt.show()


In [ ]:
y_pred_mlp2 = model_mlp2.predict(X_test)
y_pred_classes_mlp2 = np.argmax(y_pred_mlp2, axis=1)
y_true_mlp2 = np.argmax(y_test, axis=1)
accuracy_mlp2 = accuracy_score(y_true_mlp2, y_pred_classes_mlp2)
precision_mlp2 = precision_score(y_true_mlp2, y_pred_classes_mlp2, average='weighted', zero_division=0)
recall_mlp2 = recall_score(y_true_mlp2, y_pred_classes_mlp2, average='weighted')
f1_mlp2= f1_score(y_true_mlp2, y_pred_classes_mlp2, average='weighted')
results.append({'Model': 'MLP2', 'Accuracy': accuracy_mlp2, 'Precision': precision_mlp2, 'Recall': recall_mlp2, 'F1 Score': f1_mlp2})


In [ ]:
y_pred_probs = model_mlp2.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_pred_labels = decode_labels(y_pred_classes, pokemon_data)
y_true_labels = decode_labels(y_true_classes, pokemon_data)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true_labels), yticklabels=np.unique(y_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


### MLP3

In [ ]:
model_mlp3 = Sequential([
    Input(shape=(120, 120, 3)), 
    Flatten(),                
    Dense(1024),              
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),
    Dense(512),              
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),
    Dense(256),            
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.3),
    Dense(len(np.unique(pokemon_data['Type1'])), activation='softmax')
])

model_mlp3.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_mlp3.summary()
# Callback 
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-8, verbose=1)
callbacks = [early_stopping, reduce_lr]

history_mlp3 = model_mlp3.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=callbacks
)


In [ ]:
train_acc = history_mlp3.history['accuracy']
val_acc = history_mlp3.history['val_accuracy']
train_loss = history_mlp3.history['loss']
val_loss = history_mlp3.history['val_loss']

epochs = range(1, len(train_acc) + 1)

test_loss, test_acc = model_mlp3.evaluate(X_test, y_test)
print(f'Training Accuracy: {train_acc[-1]}')
print(f'Validation Accuracy: {val_acc[-1]}')
print(f'Test Accuracy: {test_acc}')
print(f'Training Loss: {train_loss[-1]}')
print(f'Validation Loss: {val_loss[-1]}')
print(f'Test Loss: {test_loss}')


In [ ]:
sns.set(style="whitegrid")

# Training and Validation Accuracy Plot
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(visible=None)
plt.show()

# Training and Validation Loss Plot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(visible=None)
plt.show()


In [ ]:
y_pred_mlp3 = model_mlp3.predict(X_test)
y_pred_classes_mlp3 = np.argmax(y_pred_mlp3, axis=1)
y_true_mlp3 = np.argmax(y_test, axis=1)
accuracy_mlp3 = accuracy_score(y_true_mlp3, y_pred_classes_mlp3)
precision_mlp3 = precision_score(y_true_mlp3, y_pred_classes_mlp3, average='weighted', zero_division=0)
recall_mlp3 = recall_score(y_true_mlp3, y_pred_classes_mlp3, average='weighted')
f1_mlp3 = f1_score(y_true_mlp3, y_pred_classes_mlp3, average='weighted')
results.append({'Model': 'MLP3', 'Accuracy': accuracy_mlp3, 'Precision': precision_mlp3, 'Recall': recall_mlp3, 'F1 Score': f1_mlp3})


In [ ]:
y_pred_probs = model_mlp3.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_pred_labels = decode_labels(y_pred_classes, pokemon_data)
y_true_labels = decode_labels(y_true_classes, pokemon_data)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true_labels), yticklabels=np.unique(y_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


### MLP4

In [ ]:
model_mlp4 = Sequential([
    Input(shape=(120, 120, 3)),
    Flatten(),    
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(y_train.shape[1], activation='softmax')
])

model_mlp4.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model_mlp4.summary()

# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-8)

history_mlp4 = model_mlp4.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=[early_stopping, reduce_lr]
)


In [ ]:
train_acc = history_mlp4.history['accuracy']
val_acc = history_mlp4.history['val_accuracy']
train_loss = history_mlp4.history['loss']
val_loss = history_mlp4.history['val_loss']

epochs = range(1, len(train_acc) + 1)

test_loss, test_acc = model_mlp4.evaluate(X_test, y_test)
print(f'Training Accuracy: {train_acc[-1]}')
print(f'Validation Accuracy: {val_acc[-1]}')
print(f'Test Accuracy: {test_acc}')
print(f'Training Loss: {train_loss[-1]}')
print(f'Validation Loss: {val_loss[-1]}')
print(f'Test Loss: {test_loss}')


In [ ]:
sns.set(style="whitegrid")

# Training and Validation Accuracy Plot
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(visible=None)
plt.show()

# Training and Validation Loss Plot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(visible=None)
plt.show()


In [ ]:
y_pred_mlp4 = model_mlp4.predict(X_test)
y_pred_classes_mlp4 = np.argmax(y_pred_mlp4, axis=1)
y_true_mlp4 = np.argmax(y_test, axis=1)
accuracy_mlp4 = accuracy_score(y_true_mlp4, y_pred_classes_mlp4)
precision_mlp4 = precision_score(y_true_mlp4, y_pred_classes_mlp4, average='weighted', zero_division=0)
recall_mlp4 = recall_score(y_true_mlp4, y_pred_classes_mlp4, average='weighted')
f1_mlp4 = f1_score(y_true_mlp4, y_pred_classes_mlp4, average='weighted')
results.append({'Model': 'MLP4', 'Accuracy': accuracy_mlp4, 'Precision': precision_mlp4, 'Recall': recall_mlp4, 'F1 Score': f1_mlp4})


In [ ]:
y_pred_probs = model_mlp4.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_pred_labels = decode_labels(y_pred_classes, pokemon_data)
y_true_labels = decode_labels(y_true_classes, pokemon_data)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true_labels), yticklabels=np.unique(y_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


## CNN Models

### CNN Basic

In [ ]:
model_cnn_basic = Sequential([
    Input(shape=(120, 120, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(18, activation='softmax')
])

model_cnn_basic.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn_basic.summary()

# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-8, verbose=1)
callbacks = [early_stopping, reduce_lr]

history_cnn_basic = model_cnn_basic.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=callbacks
)


In [ ]:
train_acc = history_cnn_basic.history['accuracy']
val_acc = history_cnn_basic.history['val_accuracy']
train_loss = history_cnn_basic.history['loss']
val_loss = history_cnn_basic.history['val_loss']

epochs = range(1, len(train_acc) + 1)

test_loss, test_acc = model_cnn_basic.evaluate(X_test, y_test)
print(f'Training Accuracy: {train_acc[-1]}')
print(f'Validation Accuracy: {val_acc[-1]}')
print(f'Test Accuracy: {test_acc}')
print(f'Training Loss: {train_loss[-1]}')
print(f'Validation Loss: {val_loss[-1]}')
print(f'Test Loss: {test_loss}')


In [ ]:
sns.set(style="whitegrid")

# Training and Validation Accuracy Plot
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(visible=None)
plt.show()

# Training and Validation Loss Plot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(visible=None)
plt.show()


In [ ]:
y_pred_cnn_basic = model_cnn_basic.predict(X_test)
y_pred_classes_cnn_basic = np.argmax(y_pred_cnn_basic, axis=1)
y_true_cnn_basic = np.argmax(y_test, axis=1)
accuracy_cnn_basic = accuracy_score(y_true_cnn_basic, y_pred_classes_cnn_basic)
precision_cnn_basic = precision_score(y_true_cnn_basic, y_pred_classes_cnn_basic, average='weighted', zero_division=0)
recall_cnn_basic = recall_score(y_true_cnn_basic, y_pred_classes_cnn_basic, average='weighted')
f1_cnn_basic = f1_score(y_true_cnn_basic, y_pred_classes_cnn_basic, average='weighted')
results.append({'Model': 'CNN Basic', 'Accuracy': accuracy_cnn_basic, 'Precision': precision_cnn_basic, 'Recall': recall_cnn_basic, 'F1 Score': f1_cnn_basic})


In [ ]:
y_pred_probs = model_cnn_basic.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_pred_labels = decode_labels(y_pred_classes, pokemon_data)
y_true_labels = decode_labels(y_true_classes, pokemon_data)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true_labels), yticklabels=np.unique(y_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


### CNN1

In [ ]:
model_cnn1 = Sequential([
    Input(shape=(120, 120, 3)),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(18, activation='softmax')
])


model_cnn1.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn1.summary()
# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-8, verbose=1)
callbacks = [early_stopping, reduce_lr]

history_cnn1 = model_cnn1.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=callbacks
)


In [ ]:
train_acc = history_cnn1.history['accuracy']
val_acc = history_cnn1.history['val_accuracy']
train_loss = history_cnn1.history['loss']
val_loss = history_cnn1.history['val_loss']

epochs = range(1, len(train_acc) + 1)

test_loss, test_acc = model_cnn1.evaluate(X_test, y_test)
print(f'Training Accuracy: {train_acc[-1]}')
print(f'Validation Accuracy: {val_acc[-1]}')
print(f'Test Accuracy: {test_acc}')
print(f'Training Loss: {train_loss[-1]}')
print(f'Validation Loss: {val_loss[-1]}')
print(f'Test Loss: {test_loss}')


In [ ]:
sns.set(style="whitegrid")

# Training and Validation Accuracy Plot
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(visible=None)
plt.show()

# Training and Validation Loss Plot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(visible=None)
plt.show()


In [ ]:
y_pred_cnn1 = model_cnn1.predict(X_test)
y_pred_classes_cnn1 = np.argmax(y_pred_cnn1, axis=1)
y_true_cnn1 = np.argmax(y_test, axis=1)
accuracy_cnn1 = accuracy_score(y_true_cnn1, y_pred_classes_cnn1)
precision_cnn1 = precision_score(y_true_cnn1, y_pred_classes_cnn1, average='weighted', zero_division=0)
recall_cnn1 = recall_score(y_true_cnn1, y_pred_classes_cnn1, average='weighted')
f1_cnn1 = f1_score(y_true_cnn1, y_pred_classes_cnn1, average='weighted')
results.append({'Model': 'CNN1', 'Accuracy': accuracy_cnn1, 'Precision': precision_cnn1, 'Recall': recall_cnn1, 'F1 Score': f1_cnn1})


In [ ]:
y_pred_probs = model_cnn1.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_pred_labels = decode_labels(y_pred_classes, pokemon_data)
y_true_labels = decode_labels(y_true_classes, pokemon_data)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true_labels), yticklabels=np.unique(y_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


### CNN2

In [ ]:
base_model = VGG16(include_top=False, input_shape=(120, 120, 3), weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False

model_cnn2 = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    #BatchNormalization(),
    Dense(18, activation='softmax')
])

model_cnn2.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-8, verbose=1)

history_cnn2 = model_cnn2.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=[early_stopping, reduce_lr]
)

model_cnn2.summary()


In [ ]:
train_acc = history_cnn2.history['accuracy']
val_acc = history_cnn2.history['val_accuracy']
train_loss = history_cnn2.history['loss']
val_loss = history_cnn2.history['val_loss']

epochs = range(1, len(train_acc) + 1)

test_loss, test_acc = model_cnn2.evaluate(X_test, y_test)
print(f'Training Accuracy: {train_acc[-1]}')
print(f'Validation Accuracy: {val_acc[-1]}')
print(f'Test Accuracy: {test_acc}')
print(f'Training Loss: {train_loss[-1]}')
print(f'Validation Loss: {val_loss[-1]}')
print(f'Test Loss: {test_loss}')


In [ ]:
sns.set(style="whitegrid")

# Training and Validation Accuracy Plot
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(visible=None)
plt.show()

# Training and Validation Loss Plot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(visible=None)
plt.show()


In [ ]:
y_pred_cnn2 = model_cnn2.predict(X_test)
y_pred_classes_cnn2 = np.argmax(y_pred_cnn2, axis=1)
y_true_cnn2 = np.argmax(y_test, axis=1)
accuracy_cnn2 = accuracy_score(y_true_cnn2, y_pred_classes_cnn2)
precision_cnn2 = precision_score(y_true_cnn2, y_pred_classes_cnn2, average='weighted', zero_division=0)
recall_cnn2 = recall_score(y_true_cnn2, y_pred_classes_cnn2, average='weighted')
f1_cnn2 = f1_score(y_true_cnn2, y_pred_classes_cnn2, average='weighted')
results.append({'Model': 'CNN2', 'Accuracy': accuracy_cnn2, 'Precision': precision_cnn2, 'Recall': recall_cnn2, 'F1 Score': f1_cnn2})


In [ ]:
y_pred_probs = model_cnn2.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_pred_labels = decode_labels(y_pred_classes, pokemon_data)
y_true_labels = decode_labels(y_true_classes, pokemon_data)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true_labels), yticklabels=np.unique(y_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


### CNN3

In [ ]:
base_model = VGG16(include_top=False, input_shape=(120, 120, 3), weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False

model_cnn3 = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.02)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.02)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(18, activation='softmax')
])

model_cnn3.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-8, verbose=1)

history_cnn3 = model_cnn3.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=[early_stopping, reduce_lr]
)

model_cnn3.summary()


In [ ]:
train_acc = history_cnn3.history['accuracy']
val_acc = history_cnn3.history['val_accuracy']
train_loss = history_cnn3.history['loss']
val_loss = history_cnn3.history['val_loss']

epochs = range(1, len(train_acc) + 1)

test_loss, test_acc = model_cnn3.evaluate(X_test, y_test)
print(f'Training Accuracy: {train_acc[-1]}')
print(f'Validation Accuracy: {val_acc[-1]}')
print(f'Test Accuracy: {test_acc}')
print(f'Training Loss: {train_loss[-1]}')
print(f'Validation Loss: {val_loss[-1]}')
print(f'Test Loss: {test_loss}')


In [ ]:
sns.set(style="whitegrid")

# Training and Validation Accuracy Plot
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(visible=None)
plt.show()

# Training and Validation Loss Plot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(visible=None)
plt.show()


In [ ]:
y_pred_cnn3 = model_cnn3.predict(X_test)
y_pred_classes_cnn3 = np.argmax(y_pred_cnn3, axis=1)
y_true_cnn3 = np.argmax(y_test, axis=1)
accuracy_cnn3 = accuracy_score(y_true_cnn3, y_pred_classes_cnn3)
precision_cnn3 = precision_score(y_true_cnn3, y_pred_classes_cnn3, average='weighted', zero_division=0)
recall_cnn3 = recall_score(y_true_cnn3, y_pred_classes_cnn3, average='weighted')
f1_cnn3 = f1_score(y_true_cnn3, y_pred_classes_cnn3, average='weighted')
results.append({'Model': 'CNN3', 'Accuracy': accuracy_cnn3, 'Precision': precision_cnn3, 'Recall': recall_cnn3, 'F1 Score': f1_cnn3})


In [ ]:
y_pred_probs = model_cnn3.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_pred_labels = decode_labels(y_pred_classes, pokemon_data)
y_true_labels = decode_labels(y_true_classes, pokemon_data)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true_labels), yticklabels=np.unique(y_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


### CNN4

In [ ]:
model_cnn4 = Sequential([
    Input(shape=(120, 120, 3)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(18, activation='softmax')
])

model_cnn4.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-8, verbose=1)

history_cnn4 = model_cnn4.fit(
    train_generator,
    validation_data=(X_val, y_val),
    epochs=200,
    callbacks=[early_stopping, reduce_lr]
)

model_cnn4.summary()


In [ ]:
train_acc = history_cnn4.history['accuracy']
val_acc = history_cnn4.history['val_accuracy']
train_loss = history_cnn4.history['loss']
val_loss = history_cnn4.history['val_loss']

epochs = range(1, len(train_acc) + 1)

test_loss, test_acc = model_cnn4.evaluate(X_test, y_test)
print(f'Training Accuracy: {train_acc[-1]}')
print(f'Validation Accuracy: {val_acc[-1]}')
print(f'Test Accuracy: {test_acc}')
print(f'Training Loss: {train_loss[-1]}')
print(f'Validation Loss: {val_loss[-1]}')
print(f'Test Loss: {test_loss}')


In [ ]:
sns.set(style="whitegrid")

# Training and Validation Accuracy Plot
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(visible=None)
plt.show()

# Training and Validation Loss Plot
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(visible=None)
plt.show()


In [ ]:
y_pred_cnn4 = model_cnn4.predict(X_test)
y_pred_classes_cnn4 = np.argmax(y_pred_cnn4, axis=1)
y_true_cnn4 = np.argmax(y_test, axis=1)
accuracy_cnn4 = accuracy_score(y_true_cnn4, y_pred_classes_cnn4)
precision_cnn4 = precision_score(y_true_cnn4, y_pred_classes_cnn4, average='weighted', zero_division=0)
recall_cnn4 = recall_score(y_true_cnn4, y_pred_classes_cnn4, average='weighted')
f1_cnn4 = f1_score(y_true_cnn4, y_pred_classes_cnn4, average='weighted')
results.append({'Model': 'CNN4', 'Accuracy': accuracy_cnn4, 'Precision': precision_cnn4, 'Recall': recall_cnn4, 'F1 Score': f1_cnn4})


In [ ]:
y_pred_probs = model_cnn4.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

def decode_labels(labels, pokemon_data):
    type_mapping = dict(enumerate(pokemon_data['Type1'].astype('category').cat.categories))
    return np.vectorize(type_mapping.get)(labels)

y_pred_labels = decode_labels(y_pred_classes, pokemon_data)
y_true_labels = decode_labels(y_true_classes, pokemon_data)

# Confusion Matrixn
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_true_labels), yticklabels=np.unique(y_true_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()


## Summary

In [ ]:
results_df = pd.DataFrame(results)
print(results_df)
